## Setting Up:

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM
import csv

import pandas as pd
import warnings
from tqdm.notebook import tqdm
import re
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
import xarray as xr
import massbalancemachine as mbm
from collections import defaultdict
import logging
import torch.nn as nn
from skorch.helper import SliceDataset
from datetime import datetime
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint
import itertools
import random
from torch.utils.data import WeightedRandomSampler, SubsetRandomSampler

import pickle 
from collections import Counter
import ast
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset


from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *
from scripts.lstm import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
color_pink = '#c51b7d'

# RGI Ids:
# Read rgi ids:
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)

vois_climate = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
]

vois_topographical = [
    "aspect_sgi",
    "slope_sgi",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]

In [ ]:
seed_all(cfg.seed)

print("Using seed:", cfg.seed)

if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Read GL data:

In [ ]:
data_glamos = getStakesData(cfg)

# Capitalize glacier names:
glacierCap = {}
for gl in data_glamos['GLACIER'].unique():
    if isinstance(gl, str):  # Ensure the glacier name is a string
        if gl.lower() == 'claridenu':
            glacierCap[gl] = 'Clariden_U'
        elif gl.lower() == 'claridenl':
            glacierCap[gl] = 'Clariden_L'
        else:
            glacierCap[gl] = gl.capitalize()
    else:
        print(f"Warning: Non-string glacier name encountered: {gl}")

# drop taelliboden and plainemorte if in there
if 'taelliboden' in data_glamos['GLACIER'].unique():
    data_glamos = data_glamos[data_glamos['GLACIER'] != 'taelliboden']
if 'plainemorte' in data_glamos['GLACIER'].unique():
    data_glamos = data_glamos[data_glamos['GLACIER'] != 'plainemorte']

print('-------------------')
print('Number of glaciers:', len(data_glamos['GLACIER'].unique()))
print('Number of winter and annual samples:', len(data_glamos))
print('Number of annual samples:',
      len(data_glamos[data_glamos.PERIOD == 'annual']))
print('Number of winter samples:',
      len(data_glamos[data_glamos.PERIOD == 'winter']))


## Input data:
### Input dataset:

In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = False
dataloader_gl = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_LSTM.csv')
data_monthly = dataloader_gl.data

data_monthly['GLWD_ID'] = data_monthly.apply(
    lambda x: mbm.data_processing.utils.get_hash(f"{x.GLACIER}_{x.YEAR}"),
    axis=1)
data_monthly['GLWD_ID'] = data_monthly['GLWD_ID'].astype(str)

dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

data_annual = dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']

# print mean and std of N_MONTHS
print('Mean number of months:', data_annual.N_MONTHS.mean())
print('Std number of months:', data_annual.N_MONTHS.std())

# same for winter
data_winter = dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']
print('Mean number of months (winter):', data_winter.N_MONTHS.mean())
print('Std number of months (winter):', data_winter.N_MONTHS.std())

## Blocking on glaciers:

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

if missing_glaciers:
    print(
        f"Warning: The following test glaciers are not in the dataset: {missing_glaciers}"
    )

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(TEST_GLACIERS)]
print('Size of monthly test data:', len(data_test))

data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
    train_glaciers)]
print('Size of monthly train data:', len(data_train))

if len(data_train) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(data_test) / len(data_train)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

# Number of annual versus winter measurements:
print('-------------\nTrain:')
print('Number of monthly winter and annual samples:', len(data_train))
print('Number of monthly annual samples:',
      len(data_train[data_train.PERIOD == 'annual']))
print('Number of monthly winter samples:',
      len(data_train[data_train.PERIOD == 'winter']))

# Same for test
data_test_annual = data_test[data_test.PERIOD == 'annual']
data_test_winter = data_test[data_test.PERIOD == 'winter']

print('Test:')
print('Number of monthly winter and annual samples:', len(data_test))
print('Number of monthly annual samples:', len(data_test_annual))
print('Number of monthly winter samples:', len(data_test_winter))

print('Total:')
print('Number of monthly rows:', len(dataloader_gl.data))
print('Number of annual rows:',
      len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']))
print('Number of winter rows:',
      len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']))

# same for original data:
print('-------------\nIn annual format:')
print('Number of annual train rows:',
      len(data_glamos[data_glamos.GLACIER.isin(train_glaciers)]))
print('Number of annual test rows:',
      len(data_glamos[data_glamos.GLACIER.isin(TEST_GLACIERS)]))

splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)

print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
test_perc = (len(test_set['df_X']) / len(train_set['df_X'])) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))
print('Size of test set:', len(test_set['df_X']))
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Size of train set:', len(train_set['df_X']))

# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']

data_test = test_set['df_X']
data_test['y'] = test_set['y']

## LSTM:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'pcsr',
    'ELEVATION_DIFFERENCE',
]
STATIC_COLS = [
    'aspect_sgi', 'slope_sgi', 'hugonnet_dhdt', 'consensus_ice_thickness',
    'millan_v'
]

HYDRO_MONTHS = [
    'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul',
    'aug', 'sep'
]
HYDRO_POS = {m: i for i, m in enumerate(HYDRO_MONTHS)}

feature_columns = MONTHLY_COLS + STATIC_COLS

### Build LSTM dataloaders:

In [ ]:
df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()


In [ ]:
# --- build train dataset from dataframe ---
ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train, MONTHLY_COLS, STATIC_COLS, HYDRO_POS, expect_target=True)

# --- make train/val loaders and get split indices (scalers fit on TRAIN, applied to whole ds) ---
train_dl, val_dl, train_idx, val_idx = ds_train.make_loaders(
    val_ratio=0.2,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# sanity checks
print("y mean/std (train):", float(ds_train.y_mean), float(ds_train.y_std))
for name, arr in [("Xm", ds_train.Xm), ("Xs", ds_train.Xs), ("y", ds_train.y)]:
    print(name, "has NaN?", bool(torch.isnan(arr).any().item()))
print("Train counts:", int(ds_train.iw[train_idx].sum()), "winter |",
      int(ds_train.ia[train_idx].sum()), "annual")
print("Val   counts:", int(ds_train.iw[val_idx].sum()), "winter |",
      int(ds_train.ia[val_idx].sum()), "annual")

# --- build test dataset and loader (reuses train scalers) ---
ds_test = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    HYDRO_POS,
    expect_target=True  # you said test has real targets
)
test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test, ds_train, batch_size=128)

In [ ]:
N_w, N_a = int(ds_train.iw.sum()), int(ds_train.ia.sum())
w_annual = N_w / max(N_a, 1)
w_annual

## Grid search:

In [ ]:
import itertools
from functools import partial


def make_capped_grid_two_heads(Fm, Fs):
    """
    Returns exactly 150 configs:
      - 144 with two_heads=True (broad coverage)
      - 6 with two_heads=False (light baseline)
    Design:
      Trunks (12): hs in {96,128,192} x nl in {1,2} x bi in {True, False}
        - if nl==1 -> dropout=0.0
        - if nl==2 -> dropout=0.1
      Static options (2): identity; 2-layer [128,64] with dropout 0.1
      Head dropout (2): {0.0, 0.2}
      Train combos (3): (lr, wd, loss) fixed tuples (see below)
      Single-head slice: 3 representative trunks, static identity only, head_dropout=0.0,
                         2 train combos.
    """

    # ---- helper to package a config ----
    def pack(trunk, static, two_heads, head_dropout, train_combo, loss_name):
        hs, nl, bi, do = trunk
        sl, sh, sd = static  # layers, hidden, static_dropout
        lr, wd = train_combo
        loss_spec = None if loss_name == "neutral" else ("weighted",
                                                         dict(w_winter=1.0,
                                                              w_annual=3.33))
        return dict(
            Fm=Fm,
            Fs=Fs,
            hidden_size=hs,
            num_layers=nl,
            bidirectional=bi,
            dropout=do,
            static_layers=sl,
            static_hidden=sh,
            static_dropout=sd,
            two_heads=two_heads,
            head_dropout=head_dropout,
            lr=lr,
            weight_decay=wd,
            clip_val=1.0,  # fixed to keep the cap
            sched_factor=0.5,
            sched_patience=6,  # fixed to keep the cap
            loss_name=loss_name,
            loss_spec=loss_spec)

    # ---- trunk set (12 total) ----
    # hs x nl x bi with pruned dropout rule
    trunks = []
    for hs in [96, 128, 192]:
        for nl in [1, 2]:
            for bi in [True, False]:
                do = 0.0 if nl == 1 else 0.1
                trunks.append((hs, nl, bi, do))
    assert len(trunks) == 12

    # ---- static options (2 total) ----
    static_id = (0, None, None)  # identity
    static_2L = (2, [128, 64], 0.1)  # two-layer MLP with modest dropout
    statics = [static_id, static_2L]

    # ---- head dropout (2) ----
    head_dropouts = [0.0, 0.2]

    # ---- training combos & loss variants ----
    # Three combos for the two-head grid:
    train_combos_th = [
        (3e-4, 0.0),  # lr, wd
        (1e-3, 1e-4),
        (3e-4, 1e-4),
    ]
    loss_names_th = ["weighted", "neutral", "weighted"]  # align length=3

    # Two combos for the single-head slice:
    train_combos_sh = [
        (3e-4, 0.0),
        (1e-3, 1e-4),
    ]
    loss_names_sh = ["weighted", "neutral"]

    grid = []

    # ---- 144 two-head configs ----
    for trunk in trunks:
        for static in statics:
            for hd in head_dropouts:
                for (lr, wd), loss_name in zip(train_combos_th, loss_names_th):
                    grid.append(
                        pack(trunk, static, True, hd, (lr, wd), loss_name))

    # ---- 6 single-head configs (light baseline) ----
    # Choose 3 representative trunks (covering nl/bidirectional variety)
    rep_trunks = [
        (128, 1, True, 0.0),
        (128, 2, True, 0.1),
        (192, 2, False, 0.1),
    ]
    for trunk, (lr, wd), loss_name in zip(
            itertools.chain.from_iterable([[t] * 2 for t in rep_trunks
                                           ]),  # repeat each trunk 2 times
            train_combos_sh * 3,
            loss_names_sh * 3):
        grid.append(pack(trunk, static_id, False, 0.0, (lr, wd), loss_name))

    assert len(grid) == 150
    return grid

def make_capped_grid_one_head(Fm, Fs):
    """
    Single-head grid (two_heads=False).
    Default: 144 configs = 12 trunks × 2 statics × 2 head_dropouts × 3 train combos.
    Optional: pad_to_150=True adds 6 extra configs → total 150.

    Notes:
      - static_id uses (0, 0, None) so it also works if your __init__ doesn't handle None for
        static_hidden; an empty width list will make the static MLP an Identity.
      - If you *did* patch __init__ to accept None for identity, you can swap 0->None safely.
    """

    def pack(trunk, static, head_dropout, train_combo, loss_name):
        hs, nl, bi, do = trunk
        sl, sh, sd = static  # layers, hidden, static_dropout
        lr, wd = train_combo
        loss_spec = None if loss_name == "neutral" else ("weighted",
                                                         dict(w_winter=1.0, w_annual=3.33))
        return dict(
            Fm=Fm, Fs=Fs,
            hidden_size=hs, num_layers=nl, bidirectional=bi, dropout=do,
            static_layers=sl, static_hidden=sh, static_dropout=sd,
            two_heads=False, head_dropout=head_dropout,
            lr=lr, weight_decay=wd, clip_val=1.0,
            sched_factor=0.5, sched_patience=6,
            loss_name=loss_name, loss_spec=loss_spec
        )

    # --- trunks (12): hs in {96,128,192} × nl in {1,2} × bi in {True,False}
    # pruning: if nl==1 → dropout=0.0; if nl==2 → dropout=0.1
    trunks = []
    for hs in [96, 128, 192]:
        for nl in [1, 2]:
            for bi in [True, False]:
                do = 0.0 if nl == 1 else 0.1
                trunks.append((hs, nl, bi, do))
    assert len(trunks) == 12

    # --- static options (2)
    static_id = (0, 0, None)          # identity (use 0 here for robustness)
    static_2L = (2, [128, 64], 0.1)   # small two-layer MLP
    statics = [static_id, static_2L]

    # --- head dropout (2) — still relevant with one head
    head_dropouts = [0.0, 0.2]

    # --- training combos (3) + loss mapping
    train_combos = [
        (3e-4, 0.0),
        (1e-3, 1e-4),
        (3e-4, 1e-4),
    ]
    loss_names = ["weighted", "neutral", "weighted"]  # align with 3 combos

    grid = []
    for trunk in trunks:
        for static in statics:
            for hd in head_dropouts:
                for (lr, wd), loss_name in zip(train_combos, loss_names):
                    grid.append(pack(trunk, static, hd, (lr, wd), loss_name))
    return grid

In [ ]:
# sampled_params = make_capped_grid(ds_train.Xm.shape[-1], ds_train.Xs.shape[-1])

sampled_params = make_capped_grid_two_heads(ds_train.Xm.shape[-1], ds_train.Xs.shape[-1])
print("Total configs:", len(sampled_params))  # should print 150
sampled_params[:20]

In [ ]:
RUN = True
if RUN:
    os.makedirs("logs", exist_ok=True)
    os.makedirs("models", exist_ok=True)

    log_filename = f'logs/two_heads_lstm_param_search_progress_{datetime.now().strftime("%Y-%m-%d")}.csv'

    # create log with header
    with open(log_filename, mode='w', newline='') as log_file:
        writer = csv.DictWriter(log_file,
                                fieldnames=list(sampled_params[0].keys()) +
                                ['valid_loss', 'test_rmse_a', 'test_rmse_w'])
        writer.writeheader()

    results = []
    best_overall = {"val": float('inf'), "row": None, "params": None}

    for i, params in enumerate(sampled_params):
        seed_all(cfg.seed)
        print(f"\n--- Running config {i+1}/{len(sampled_params)} ---")
        print(params)

        # Build model
        model = mbm.models.LSTM_MB.build_model_from_params(params, device)

        # Choose loss
        loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(params)

        # Train
        history, best_val, best_state = model.train_loop(
            device=device,
            train_dl=train_dl,
            val_dl=val_dl,
            epochs=150,
            lr=params['lr'],
            weight_decay=params['weight_decay'],
            clip_val=params['clip_val'],
            # scheduler
            sched_factor=params['sched_factor'],
            sched_patience=params['sched_patience'],
            sched_threshold=0.01,
            sched_threshold_mode="rel",
            sched_cooldown=1,
            sched_min_lr=1e-6,
            # early stopping
            es_patience=15,
            es_min_delta=1e-4,
            # logging
            log_every=5,
            verbose=False,
            # checkpoint
            save_best_path="models/two_heads_best_lstm_mb_gs.pt",
            loss_fn=loss_fn,
        )

        # Load the best weights
        best_state = torch.load("models/two_heads_best_lstm_mb_gs.pt",
                                map_location=device)
        model.load_state_dict(best_state)

        # Evaluate on test
        test_metrics, test_df_preds = model.evaluate_with_preds(
            device, test_dl, ds_test)
        test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
            'RMSE_winter']

        # Log row
        row = {
            **params, 'valid_loss': float(best_val),
            'test_rmse_a': float(test_rmse_a),
            'test_rmse_w': float(test_rmse_w)
        }
        with open(log_filename, mode='a', newline='') as log_file:
            writer = csv.DictWriter(log_file, fieldnames=list(row.keys()))
            writer.writerow(row)

        results.append(row)

        # Track best by validation loss
        if best_val < best_overall['val']:
            best_overall = {"val": best_val, "row": row, "params": params}

    print("\n=== Best config by validation loss ===")
    print(best_overall['params'])
    print(best_overall['row'])
